# Earth System Simulation - Google Colab Version

This notebook shows how to run the Earth system simulation with proper gradient handling.

In [ ]:
# Install required packages
!pip install torch numpy matplotlib pyyaml tqdm plotly

# Clone repository
!git clone https://github.com/yourusername/earth_system_sim.git
%cd earth_system_sim

# Install package
!pip install -e .

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import sys
from typing import Dict, List, Tuple, Optional
from tqdm.notebook import tqdm
import contextlib

# Add project root to Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from scripts.run_simulation import EarthSystemSimulation
from scripts.visualize_results import create_visualizations

In [ ]:
class NoGradSimulation:
    """Wrapper class to ensure simulation runs without gradients."""
    
    def __init__(self, sim: EarthSystemSimulation):
        self.sim = sim
        
    def run_with_progress(self, num_steps: int, save_frequency: int) -> Dict:
        """Run simulation with progress tracking and guaranteed no gradients."""
        try:
            print("Starting simulation...")
            
            # Storage for trajectory
            trajectory = {
                'physical': [],
                'biosphere': [],
                'geosphere': [],
                'times': []
            }
            
            # Ensure no gradients throughout simulation
            with torch.no_grad():
                # Initialize states
                physical_state, biosphere_state, geosphere_state = self.sim._initialize_states()
                
                # Save initial states
                trajectory['physical'].append(physical_state.cpu().numpy())
                trajectory['biosphere'].append(biosphere_state.cpu().numpy())
                trajectory['geosphere'].append(geosphere_state.cpu().numpy())
                trajectory['times'].append(self.sim.synchronizer.current_times)
                
                # Run simulation with progress bar
                for step in tqdm(range(num_steps), desc='Simulation Progress'):
                    # Run timestep
                    physical_state, biosphere_state, geosphere_state = self.sim.run_timestep(
                        physical_state, biosphere_state, geosphere_state
                    )
                    
                    # Save states periodically
                    if step % save_frequency == 0:
                        trajectory['physical'].append(physical_state.cpu().numpy())
                        trajectory['biosphere'].append(biosphere_state.cpu().numpy())
                        trajectory['geosphere'].append(geosphere_state.cpu().numpy())
                        trajectory['times'].append(self.sim.synchronizer.current_times)
            
            print("Simulation completed successfully!")
            return trajectory
        
        except Exception as e:
            print(f"Error during simulation: {str(e)}")
            raise

In [ ]:
# Load and validate configuration
with open('config/model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Adjust for Colab resources
config['grid_height'] = 16
config['grid_width'] = 16

# Save modified config
with open('config/colab_config.yaml', 'w') as f:
    yaml.dump(config, f)

# Initialize simulation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

sim = EarthSystemSimulation('config/colab_config.yaml', device)
wrapped_sim = NoGradSimulation(sim)

# Run simulation with no gradients
trajectory = wrapped_sim.run_with_progress(num_steps=50, save_frequency=5)

In [ ]:
def plot_results(trajectory: Dict):
    """Plot simulation results."""
    fig, axes = plt.subplots(3, 1, figsize=(12, 12))
    
    # Plot physical system results
    temp_mean = np.mean([state[..., 1] for state in trajectory['physical']], axis=(1, 2))
    axes[0].plot(temp_mean, 'r-', label='Mean Temperature')
    axes[0].set_title('Physical System')
    axes[0].grid(True)
    axes[0].legend()
    
    # Plot biosphere results
    veg_mean = np.mean([state[..., 0] for state in trajectory['biosphere']], axis=1)
    axes[1].plot(veg_mean, 'g-', label='Mean Vegetation')
    axes[1].set_title('Biosphere System')
    axes[1].grid(True)
    axes[1].legend()
    
    # Plot geosphere results
    elev_mean = np.mean([state[..., 0] for state in trajectory['geosphere']], axis=1)
    axes[2].plot(elev_mean, 'b-', label='Mean Elevation')
    axes[2].set_title('Geosphere System')
    axes[2].grid(True)
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

# Plot results
plot_results(trajectory)

In [ ]:
# Save results to file
try:
    from google.colab import drive
    drive.mount('/content/drive')
    save_dir = '/content/drive/MyDrive/earth_system_results'
except:
    save_dir = 'results'
    
import os
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'simulation_results.npz')

np.savez(
    save_path,
    physical_states=np.array(trajectory['physical']),
    biosphere_states=np.array(trajectory['biosphere']),
    geosphere_states=np.array(trajectory['geosphere']),
    times=trajectory['times']
)

print(f"Results saved to: {save_path}")